<h1><center>P7 - CREATE & IMPLEMENT A CREDIT SCORING MODEL</center></h1>

### OVERVIEW

As a Data Scientist within the financial institution "Prêt à dépenser" which give loans to people with insufficient or non-existent credit history.

Logo entreprise 

The company wishes to implement a credit scoring model in order to make good decisions.
Two types of risks are associated with the bank’s decision:

a. If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

b. If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company.

In addition, customers service are pushed more and more by clients to explain their decision to grant a loan and it's in this mindset that "Prêt à dépenser" wants to move forward.

"Prêt à dépenser" has then decided to develop an interactive dashboard so that not only their customers service team can explain - in the most transparent way - their decisions to give or not the loans to their clients but also to give full access to their own information. 

### DATA

The dataset provided contains a vast number of details about the borrower. It is separated into several relational tables, which contain applicants’ static data such as their gender, age, number of family members, occupation, and other necessary fields, applicant’s previous credit history obtained from the credit bureau department, and the applicant’s past credit history.<br>
Data can be found here:<br> https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip


### OBJECTIVE

* Create & implement a credit scoring model which will return automatically a probability for a client to default his loan.
* Create an interactive dashboard to later provide to the customers service management team so that they can interpret the model's predictions. It will provide them valuable information in order to improve their knowledge of the clients and to train their staff.

In order to mainly focus on the creation, implementation and optimization of our model, Michaël, our manager, encouraged us to carefully choose an existing Kaggle kernel to ease the pre-processing work on this dataset.<br>
We decided to opt for the kernel of student Rishabh Rao who did an exhaustive data pre-processing work.

https://www.kaggle.com/rishabhrao/home-credit-default-risk-extensive-eda 

###  Dashboard constraints

Michaël provided us specifications for the interactive dashboard. It will at least need to contain the followings features:

* Easy viewing of the score and its interpretation for each client (especially for non scientific people).
* Easy viewing of the descriptive information of the clients (via a filter system).
* Allow the user to compare descriptive informations of one client to the others or to a group of similar clients.

### Livrables 

Le dashboard interactif répondant aux spécifications ci-dessus et l’API de prédiction du score, déployées chacunes sur le cloud.
Un dossier sur un outil de versioning de code contenant :
Le code de la modélisation (du prétraitement à la prédiction)
Le code générant le dashboard
Le code permettant de déployer le modèle sous forme d'API
Une note méthodologique décrivant :
La méthodologie d'entraînement du modèle (2 pages maximum)
La fonction coût métier, l'algorithme d'optimisation et la métrique d'évaluation (1 page maximum)
L’interprétabilité globale et locale du modèle (1 page maximum)
Les limites et les améliorations possibles (1 page maximum)


## PART 2 - Feature Engineering

## Table of Contents

* [1. Dataset Overview](#chapter1)
* [2. EDA of input files](#chapter2)
    * [2.1 application_train/test](#section_2_1)
    * [2.2 bureau.csv](#section_2_2)
    * [2.3 bureau_balance.csv](#section_2_3)
    * [2.4 previous_application.csv](#section_2_4)
    * [2.5 installments_payments.csv](#section_2_5)
    * [2.6 POS_CASH_balance.csv](#section_2_6)
    * [2.7 credit_card_balance.csv](#section_2_7)
* [3. EDA Conclusions](#chapter3)

In [5]:
# ! usr/bin/env python 3
# coding: utf-8

# Most features are created by applying min, max, mean, sum and var functions to grouped tables. 
# Little feature selection is done and overfitting might be a problem since many features are related.
# The following key ideas were used:
# - Divide or subtract important features to get rates (like annuity and income)
# - In Bureau Data: create specific features for Active credits and Closed credits
# - In Previous Applications: create specific features for Approved and Refused applications
# - Modularity: one function for each table (except bureau_balance and application_test)
# - One-hot encoding for categorical features
# All tables are joined with the application DF using the SK_ID_CURR key (except bureau_balance).
# You can use LightGBM with KFold or Stratified KFold.


import numpy as np
import pandas as pd
import gc
import re
import time
from datetime import datetime
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
from functions_P7_feat import*
# from LightGBM import*
# from LightGBM_extra import*
palette = sns.color_palette("bright", 10)
plot_kwds = {'alpha' : 1, 's' : 60, 'linewidths':0}
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All functions run in this notebook can be found at: functions_P7_feat.py

In [6]:
start_time = datetime.now()

In [7]:
# from google.colab import files
# uploaded = files.upload()

In [8]:
# Transforming our csv files into dataframe
# %%time
homecredit = pd.read_csv('HomeCredit_columns_description.csv', index_col=[0])
cash_balance = pd.read_csv('POS_CASH_balance.csv')
app_test = pd.read_csv('application_test.csv')
app_train = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
cc_balance = pd.read_csv('credit_card_balance.csv')
inst_payments = pd.read_csv('installments_payments.csv')
previous_app = pd.read_csv('previous_application.csv')
sample_submission = pd.read_csv('sample_submission.csv')

## 1. Dataset Overview<a class="anchor" id="chapter1"></a>

In [9]:
# INITIALISE FILES_SPEC TO SERIES DICTIONARY

list_files = ["HomeCredit_columns_description.csv",
              "POS_CASH_balance.csv",
              "application_test.csv",
              "application_train.csv",
              "bureau.csv",
              "bureau_balance.csv",
              "credit_card_balance.csv",
              "installments_payments.csv",
              "previous_application.csv",
              "sample_submission.csv"]

files_spec = {
             'Total rows': pd.Series([len(homecredit),
                                       len(cash_balance),
                                       len(app_test),
                                       len(app_train),
                                       len(bureau),
                                       len(bureau_balance),
                                       len(cc_balance),
                                       len(inst_payments),
                                       len(previous_app),
                                       len(sample_submission)],
                                      index=list_files),
              'Total columns': pd.Series([len(homecredit.columns),
                                          len(cash_balance.columns),
                                          len(app_test.columns),
                                          len(app_train.columns),
                                          len(bureau.columns),
                                          len(bureau_balance.columns),
                                          len(cc_balance.columns),
                                          len(inst_payments.columns),
                                          len(previous_app.columns),
                                          len(sample_submission.columns)],
                                         index=list_files),
              'Total duplicates': pd.Series([duplicates(homecredit),
                                             duplicates(cash_balance),
                                             duplicates(app_test),
                                             duplicates(app_train),
                                             duplicates(bureau),
                                             duplicates(bureau_balance),
                                             duplicates(cc_balance),
                                             duplicates(inst_payments),
                                             duplicates(previous_app),
                                             duplicates(sample_submission)],
                                            index=list_files),
              'Total NaN values': pd.Series([missing_cells(homecredit),
                                                missing_cells(cash_balance),
                                                missing_cells(app_test),
                                                missing_cells(app_train),
                                                missing_cells(bureau),
                                                missing_cells(bureau_balance),
                                                missing_cells(cc_balance),
                                                missing_cells(inst_payments),
                                                missing_cells(previous_app),
                                                missing_cells(sample_submission)],
                                               index=list_files),
               'NaN values (%)': pd.Series([missing_cells_percent(homecredit),
                                            missing_cells_percent(cash_balance),
                                            missing_cells_percent(app_test),
                                            missing_cells_percent(app_train),
                                            missing_cells_percent(bureau),
                                            missing_cells_percent(bureau_balance),
                                            missing_cells_percent(cc_balance),
                                            missing_cells_percent(inst_payments),
                                            missing_cells_percent(previous_app),
                                            missing_cells_percent(sample_submission)],
                                           index=list_files)}

# Creates Dataframe df_files_spec

df_files_spec = pd.DataFrame(files_spec)
index = df_files_spec.index
index.name = "Input File Name"
round(df_files_spec, 2)

,Total rows,Total columns,Total duplicates,Total NaN values,NaN values (%)
Input File Name,,,,,
HomeCredit_columns_description.csv,219,4,0,133,15.18
POS_CASH_balance.csv,10001358,8,0,52158,0.07
application_test.csv,48744,121,0,1404419,23.81
application_train.csv,307511,122,0,9152465,24.40
bureau.csv,1716428,17,0,3939947,13.50
bureau_balance.csv,27299925,3,0,0,0.00
credit_card_balance.csv,3840312,23,0,5877356,6.65
installments_payments.csv,13605401,8,0,5810,0.01
previous_application.csv,1670214,37,0,11109336,17.98


<h2><center>Relationship between all tables</center></h2>

<div style="width:100%;text-align: center;"> <img align=middle src="https://miro.medium.com/max/875/0*IMyhw8RGEoDhC7t1.png" alt="Heat beating" style="height:400px;margin-top:3rem;"> </div><br>

## 2. Input files processing<a class="anchor" id="chapter2"></a>

In [10]:
# --------------------------------------------------------------------
# -- Function 1 
# --------------------------------------------------------------------

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [11]:
# --------------------------------------------------------------------
# -- Function 2 
# --------------------------------------------------------------------

def reduce_memory(df):
    """Reduce memory usage of a dataframe by setting data types. """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                # Can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df


#### 2.1 application_{train/test}

In [12]:
# --------------------------------------------------------------------
# -- Function 3 
# --------------------------------------------------------------------

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('application_train.csv', nrows= num_rows)
    test_df = pd.read_csv('application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return reduce_memory(df)

#### 2.2 bureau & bureau_balance

In [13]:
# --------------------------------------------------------------------
# -- Function 3 
# --------------------------------------------------------------------

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('bureau.csv', nrows = num_rows)
    bb = pd.read_csv('bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'MONTHS_BALANCE_MIN': ['min'], # bureau_balance
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'], 
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'], # bureau
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'AMT_ANNUITY': ['max', 'mean']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    bureau_agg = bureau_agg.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    del closed, closed_agg, bureau
    gc.collect()
    return reduce_memory(bureau_agg)

In [14]:
# --------------------------------------------------------------------
# -- Function 3
# --------------------------------------------------------------------

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    prev_agg = prev_agg.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return reduce_memory(prev_agg)

In [15]:
# --------------------------------------------------------------------
# -- Function 4
# --------------------------------------------------------------------

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    pos_agg = pos_agg.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    del pos
    gc.collect()
    return reduce_memory(pos_agg)

In [16]:
# --------------------------------------------------------------------
# -- Function 5 
# --------------------------------------------------------------------

# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    ins_agg = ins_agg.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    del ins
    gc.collect()
    return reduce_memory(ins_agg)

In [17]:
# --------------------------------------------------------------------
# -- Function 6 
# --------------------------------------------------------------------
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    cc_agg = cc_agg.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    del cc
    gc.collect()
    return reduce_memory(cc_agg)

In [18]:
application_train_test(10000)

Train samples: 10000, test samples: 10000
Initial df memory usage is 20.20 MB for 246 columns
Final memory usage is: 9.02 MB - decreased by 55.3%


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,1.0,0.0,1.0,0.0,0.067322,0.498047,202500.0,0.121948,0.060760
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,1.0,0.0,0.070862,0.208740,135000.0,0.132202,0.027603
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.011810,0.500000,67500.0,0.099976,0.049988
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.159912,0.431641,67500.0,0.219849,0.094971
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.152466,0.236816,121500.0,0.179932,0.042633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9995,172551,NaN,1,0,0,0,135000.0,454500.0,29173.5,...,0.0,1.0,0.0,1.0,0.0,0.614258,0.297119,135000.0,0.216064,0.064209
19996,9996,172556,NaN,0,1,1,1,180000.0,500490.0,52555.5,...,0.0,0.0,0.0,1.0,0.0,0.141235,0.359619,60000.0,0.291992,0.104980
19997,9997,172562,NaN,1,0,0,0,202500.0,523152.0,37336.5,...,1.0,0.0,0.0,1.0,0.0,0.006618,0.386963,202500.0,0.184326,0.071350
19998,9998,172570,NaN,0,0,0,1,382500.0,967500.0,31338.0,...,0.0,1.0,0.0,1.0,0.0,0.056488,0.395264,127500.0,0.081909,0.032379


In [19]:
# --------------------------------------------------------------------
# -- Function 9 
# --------------------------------------------------------------------
    
def merged_all(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("-------------------------------")
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("-------------------------------")
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
        
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("-------------------------------")
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("-------------------------------")
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()

    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("-------------------------------")
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
        
        return df
df = merged_all()
print('Final dataset shape:', df.shape)

Train samples: 307511, test samples: 48744
Initial df memory usage is 360.47 MB for 248 columns
Final memory usage is: 162.74 MB - decreased by 54.9%
Initial df memory usage is 266.85 MB for 116 columns
Final memory usage is: 92.45 MB - decreased by 65.4%
-------------------------------
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 23s
Initial df memory usage is 614.65 MB for 249 columns
Final memory usage is: 199.71 MB - decreased by 67.5%
-------------------------------
Previous applications df shape: (338857, 249)
Process previous_applications - done in 35s
Initial df memory usage is 46.64 MB for 18 columns
Final memory usage is: 13.83 MB - decreased by 70.3%
-------------------------------
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 16s
Initial df memory usage is 69.95 MB for 26 columns
Final memory usage is: 34.00 MB - decreased by 51.4%
-------------------------------
Installments payments df shape: (339587, 26)
Process 

In [22]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
print('Final shape for train_df:', train_df.shape)
print('Final shape for test_df:', test_df.shape)
train_df.to_pickle('./train_data2.pkl')
test_df.to_pickle('./test_data2.pkl')

Final shape for train_df: (307507, 798)
Final shape for test_df: (48744, 798)


In [21]:
elapsed = datetime.now() - start_time
print(f'Running time (min) of EDA: {elapsed}')

Running time (min) of EDA: 0:04:01.590449
